In [ ]:
import time
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [ ]:
#***************************************************Q1***************************************************

In [ ]:
oil = pd.read_csv('oil.csv')
print(oil) #before the update the non-null

oil['dcoilwtico'] = oil['dcoilwtico'].fillna(oil['dcoilwtico'].mean())

print('\n\n')
print(oil) #after the update the non-null

            date  dcoilwtico
0     2013-01-01         NaN
1     2013-01-02       93.14
2     2013-01-03       92.97
3     2013-01-04       93.12
4     2013-01-07       93.20
...          ...         ...
1213  2017-08-25       47.65
1214  2017-08-28       46.40
1215  2017-08-29       46.46
1216  2017-08-30       45.96
1217  2017-08-31       47.26

[1218 rows x 2 columns]



            date  dcoilwtico
0     2013-01-01   67.714366
1     2013-01-02   93.140000
2     2013-01-03   92.970000
3     2013-01-04   93.120000
4     2013-01-07   93.200000
...          ...         ...
1213  2017-08-25   47.650000
1214  2017-08-28   46.400000
1215  2017-08-29   46.460000
1216  2017-08-30   45.960000
1217  2017-08-31   47.260000

[1218 rows x 2 columns]


In [ ]:
#***************************************************Q2***************************************************

In [ ]:
train = pd.read_csv('train.csv')
augmented_train = pd.merge(train, oil, on = 'date')
augmented_train['dcoilwtico'] = augmented_train['dcoilwtico'].fillna(oil['dcoilwtico'].mean())
print('***********************************************augmented_train*******************************************')
print(augmented_train)

print('\n')

test = pd.read_csv('test.csv')
augmented_test= pd.merge(test, oil, on = 'date')
augmented_test['dcoilwtico'] = augmented_test['dcoilwtico'].fillna(oil['dcoilwtico'].mean())
print('***********************************************augmented_test*******************************************')
print(augmented_test)

***********************************************augmented_train*******************************************
              id        date  store_nbr                      family     sales  \
0              0  2013-01-01          1                  AUTOMOTIVE     0.000   
1              1  2013-01-01          1                   BABY CARE     0.000   
2              2  2013-01-01          1                      BEAUTY     0.000   
3              3  2013-01-01          1                   BEVERAGES     0.000   
4              4  2013-01-01          1                       BOOKS     0.000   
...          ...         ...        ...                         ...       ...   
2143741  3000883  2017-08-15          9                     POULTRY   438.133   
2143742  3000884  2017-08-15          9              PREPARED FOODS   154.553   
2143743  3000885  2017-08-15          9                     PRODUCE  2419.729   
2143744  3000886  2017-08-15          9  SCHOOL AND OFFICE SUPPLIES   121.000   
214

In [ ]:
#***************************************************Q3***************************************************

In [ ]:
#get a copy of table train with just the numerical columns hence dropping date, and family columns
#to see correlations hence determine strongest predictor
train_numerical = train.drop(['date', 'family'], axis=1)

corr_matrix = train_numerical.corr()
corr_matrix['sales'].sort_values(ascending=False)

sales          1.000000
onpromotion    0.755648
id             0.085784
store_nbr      0.041196
Name: sales, dtype: float64

In [ ]:
#----------------------------------------using linear regression first----------------------------------------
#strongest predictor is 'onpromotion' therefore will be used to predict sales
X_train = train['onpromotion'].values.reshape(-1, 1)
y_train = train['sales']

model = LinearRegression()
model.fit(X_train, y_train)

# Use the trained model to make predictions on test feature
X_test = test['onpromotion'].values.reshape(-1, 1)
predicted_sales1 = model.predict(X_test)

#saving results
predicted_sales_lr = pd.DataFrame({'Predicted_Sales': predicted_sales1})
test_with_predicted_sales1 = pd.concat([test, predicted_sales_lr], axis=1)
test_with_predicted_sales1.to_csv('test_with_predicted_sales_lr.csv', index=False)
print(test_with_predicted_sales1.head())

        id        date  store_nbr      family  onpromotion  Predicted_Sales
0  3000888  2017-08-16          1  AUTOMOTIVE         20.0       235.541344
1  3000889  2017-08-16          1   BABY CARE          1.0        76.500624
2  3000890  2017-08-16          1      BEAUTY          8.0       135.094573
3  3000891  2017-08-16          1   BEVERAGES        560.0      4755.646021
4  3000892  2017-08-16          1       BOOKS          1.0        76.500624


In [ ]:
#----------------------------------------using random forest regressor----------------------------------------
#strongest predictor is 'onpromotion' therefore will be used to predict sales
X_train = train['onpromotion'].values.reshape(-1, 1)
y_train = train['sales']

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Use the trained model to make predictions on test feature
X_test = test['onpromotion'].values.reshape(-1, 1)
predicted_sales2 = model.predict(X_test)

#saving results
predicted_sales_rfr = pd.DataFrame({'Predicted_Sales': predicted_sales2})
test_with_predicted_sales2 = pd.concat([test, predicted_sales_rfr], axis=1)
test_with_predicted_sales2.to_csv('test_with_predicted_sales_rfr.csv', index=False)
print(test_with_predicted_sales2.head())

        id        date  store_nbr      family  onpromotion  Predicted_Sales
0  3000888  2017-08-16          1  AUTOMOTIVE         20.0       195.271810
1  3000889  2017-08-16          1   BABY CARE          1.0         2.362817
2  3000890  2017-08-16          1      BEAUTY          8.0        35.224084
3  3000891  2017-08-16          1   BEVERAGES        560.0      3813.764386
4  3000892  2017-08-16          1       BOOKS          1.0         2.362817


In [ ]:
#***************************************************Q4***************************************************

In [ ]:
#comparison using Root Mean Square Error (RMSE)
submission = pd.read_csv('submission.csv')
test_with_predicted_sales_lr = pd.read_csv('test_with_predicted_sales_lr.csv')
test_with_predicted_sales_rfr = pd.read_csv('test_with_predicted_sales_rfr.csv')

df = pd.DataFrame({'Squared_Residual_lr','Squared_Residual_rfr'})
# Calculate residuals for both models
df['Squared_Residual_lr'] = (submission['sales'] - test_with_predicted_sales_lr['Predicted_Sales'])**2
df['Squared_Residual_rfr'] = (submission['sales'] - test_with_predicted_sales_rfr['Predicted_Sales'])**2

# Calculate mean squared error (MSE) for both models
MSE_lr = df['Squared_Residual_lr'].mean()
MSE_rfr = df['Squared_Residual_rfr'].mean()

# Calculate RMSE for both models (by taking square root of MSE)
RMSE_linear_regression = np.sqrt(MSE_lr)
RMSE_random_forest_regressor = np.sqrt(MSE_rfr)

print('\nRMSE when linear regressions was used: ', RMSE_linear_regression)
print('\nRMSE when random forest regressor was used: ', RMSE_random_forest_regressor)


RMSE when linear regressions was used:  172.2214919266772

RMSE when random forest regressor was used:  135.04104001573702


In [ ]:
#comparison using Mean absolute deviation
df = pd.DataFrame({'Residual_lr','Abs_Residual_rfr'})
# Calculate residuals for both models
df['Residual_lr'] = submission['sales'] - test_with_predicted_sales_lr['Predicted_Sales']
df['Residual_rfr'] = submission['sales'] - test_with_predicted_sales_rfr['Predicted_Sales']

# Calculate absolute deviations
df['Abs_Residual_lr'] = abs(df['Residual_lr'])
df['Abs_Residual_rfr'] = abs(df['Residual_rfr'])

# Calculate MAD for both models
MAD_linear_regression = df['Abs_Residual_lr'].mean()
MAD_random_forest_regressor = df['Abs_Residual_rfr'].mean()

print('\nMAD when linear regressions was used: ', MAD_linear_regression)
print('\nMAD when random forest regressor was used: ', MAD_random_forest_regressor)


MAD when linear regressions was used:  153.86620519481278

MAD when random forest regressor was used:  96.66253519449704


In [ ]:
#comparison using R-squared (R²) or Coefficient of Determination

# Calculate r2 score for both models
r2_linear_regression = r2_score(submission['sales'], test_with_predicted_sales_lr['Predicted_Sales'])
r2_random_forest_regressor = r2_score(submission['sales'], test_with_predicted_sales_rfr['Predicted_Sales'])

print('\nf1 score when linear regressions was used: ', r2_linear_regression)
print('\nf1 score when random forest regressor was used: ', r2_random_forest_regressor)


f1 score when linear regressions was used:  -0.37017424412687694

f1 score when random forest regressor was used:  -5.704900310803029


ANALYSIS: all the different comparison methods show that random forest regressor as a model performs better than linear regression at predicting 'sales' by just knowing the 'onpromotion' which was the strongest predictor from all the other attributes.